<a href="https://colab.research.google.com/github/klklgooa/MachineLearning/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트
모델을 만들고 나서 테스트 세트는 딱 한 번 사용 되는 것이 좋음 따라서 추가로 검증 세트를 또 나눔.

대략 훈련세트 60%, 검증 세트 20%, 테스트 세트 20%

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


훈련 세트와 검증 세트의 평가 점수를 이용하여 최적화.
최적화 완료 후 테스트 세트로 모델을 최종평가

In [ ]:
print(dt.score(test_input, test_target))

0.8569230769230769


## 교차 검증

검증 세트를 떼어 내어 평가하는 과정을 여러번 반복함



In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target) # dt==모델객체
print(scores) #test_score.

{'fit_time': array([0.00767732, 0.00731564, 0.0097971 , 0.00756764, 0.00714135]), 'score_time': array([0.00117803, 0.00108886, 0.00125885, 0.00107193, 0.00105882]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score'])) #test_score의 평균값

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) # 분할기 사용.
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))
print(scores)

0.8574181117533719
{'fit_time': array([0.011729  , 0.00847769, 0.00865054, 0.01002097, 0.00800323,
       0.00978684, 0.0120151 , 0.00814056, 0.00813699, 0.00796819]), 'score_time': array([0.0011344 , 0.00102019, 0.00101662, 0.00103712, 0.00105262,
       0.00130534, 0.00112486, 0.00098252, 0.00095391, 0.0009973 ]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}


## 하이퍼파라미터 튜닝

모델 파라미터: 모델 훈련 시 ai가 자동으로 찾아주는...
모델이 학습할 수 없어서 사용자가 지정해야 하는 파라미터.

ex max_depth, min_samples-split, min_impurity_decrease 등..


그리드 서치
하이퍼파라미터 탐색과 교차 검증을 한 번에 수행
즉, cross_validate() 함수를 호출할필요가 없음.

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} #조건 5개, 한 조건당 5번 훈련.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # n_jobs느 병렬실행에서 사용할 cpu코어의 수. -1은 모든코어 사용
print(gs)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})


In [ ]:
gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), #9번
          'max_depth': range(5, 20, 1), #15번
          'min_samples_split': range(2, 100, 10) #10번
          } # 1350개, 5-폴드 교차 검증을 수행하므로 만들어지는 모델의 수는 6750개.

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.892053107562055
0.8615384615384616


그리드 서치를 이용한 최적화 과정 정리
1.탐색할 매개변수를 지정.
2.훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾음.
3.그리드 서치는 최상의 매개변수에서

### 랜덤 서치

scipy
파이썬의 핵심 과학 라이브러리 중 하나

state서브 패키지 사용
통계학 과 관련된 패키지.

주어진 범위에서 고르게 값을 뽑음->균등 분포에서 샘플링한다라고함
randint는정수값을 uniform은 실수값을 선택


rvs:random variates

In [ ]:
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0, 10)
rgen.rvs(10)

array([3, 0, 8, 3, 1, 7, 2, 7, 6, 2])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([105, 106,  87,  89, 104, 110, 108, 102, 101,  88]))

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.56482204, 0.18378546, 0.62099616, 0.57791477, 0.82893682,
       0.72402616, 0.53711977, 0.14129556, 0.24647056, 0.17661971])

min_samples_leaf 리프노드가 되기 위한 최소 샘플 수.

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

dt = DecisionTreeClassifier(random_state=42)
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bc0bf88d1b0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7bc0bfa432b0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bc0bf88f970>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7bc0bf88dde0>},
                   random_state=42)

In [ ]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt = gs.best_estimator_
print(dt)
print(dt.score(test_input, test_target))

DecisionTreeClassifier(max_depth=39,
                       min_impurity_decrease=0.00034102546602601173,
                       min_samples_leaf=7, min_samples_split=13,
                       random_state=42)
0.86


주의. 그리드서치,... 등은 모델을 훈련하는 것이 아니라 검증.
적절한 하이퍼파라미터를 찾는데 사용해야함.

그리드서치, 랜덤서치 실습.

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'min_impurity_decrease': 0.0001}
0.8681929740134745
0.8653846153846154


## 확인문제

In [ ]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 5 is smaller than n_iter=100. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'min_impurity_decrease': [0.0001,
                                                                  0.0002,
                                                                  0.0003,
                                                                  0.0004,
                                                                  0.0005]},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'min_impurity_decrease': 0.0001}
0.8537617531650256
0.8461538461538461
